In [ ]:
# transmission around a 90-degree waveguide bend in 2d
import matplotlib.pyplot as plt
import numpy as np

import meep as mp

resolution = 10  # pixels/um

sx = 16  # size of cell in X direction
sy = 32  # size of cell in Y direction
cell = mp.Vector3(sx, sy, 0)

dpml = 1.0
pml_layers = [mp.PML(dpml)]

pad = 4  # padding distance between waveguide and cell edge
w = 1  # width of waveguide

wvg_xcen = 0.5 * (sx - w - 2 * pad)  # x center of vert. wvg
wvg_ycen = -0.5 * (sy - w - 2 * pad)  # y center of horiz. wvg

geometry = [
    mp.Block(
        size=mp.Vector3(mp.inf, w, mp.inf),
        center=mp.Vector3(0, wvg_ycen, 0),
        material=mp.Medium(epsilon=12),
    )
]

fcen = 0.15  # pulse center frequency
df = 0.1  # pulse width (in frequency)
sources = [
    mp.Source(
        mp.GaussianSource(fcen, fwidth=df),
        component=mp.Ez,
        center=mp.Vector3(-0.5 * sx + dpml, wvg_ycen, 0),
        size=mp.Vector3(0, w, 0),
    )
]

sim = mp.Simulation(
    cell_size=cell,
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=sources,
    resolution=resolution,
)

nfreq = 100  # number of frequencies at which to compute flux

# reflected flux
refl_fr = mp.FluxRegion(
    center=mp.Vector3(-0.5 * sx + dpml + 0.5, wvg_ycen, 0), size=mp.Vector3(0, 2 * w, 0)
)
refl = sim.add_flux(fcen, df, nfreq, refl_fr)

# transmitted flux
tran_fr = mp.FluxRegion(
    center=mp.Vector3(0.5 * sx - dpml, wvg_ycen, 0), size=mp.Vector3(0, 2 * w, 0)
)
tran = sim.add_flux(fcen, df, nfreq, tran_fr)

pt = mp.Vector3(0.5 * sx - dpml - 0.5, wvg_ycen)

sim.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, pt, 1e-3))

# for normalization run, save flux fields data for reflection plane
straight_refl_data = sim.get_flux_data(refl)

# save incident power for transmission plane
straight_tran_flux = mp.get_fluxes(tran)

sim.reset_meep()

geometry = [
    mp.Block(
        mp.Vector3(sx - pad, w, mp.inf),
        center=mp.Vector3(-0.5 * pad, wvg_ycen),
        material=mp.Medium(epsilon=12),
    ),
    mp.Block(
        mp.Vector3(w, sy - pad, mp.inf),
        center=mp.Vector3(wvg_xcen, 0.5 * pad),
        material=mp.Medium(epsilon=12),
    ),
]

sim = mp.Simulation(
    cell_size=cell,
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=sources,
    resolution=resolution,
)

# reflected flux
refl = sim.add_flux(fcen, df, nfreq, refl_fr)

tran_fr = mp.FluxRegion(
    center=mp.Vector3(wvg_xcen, 0.5 * sy - dpml - 0.5, 0), size=mp.Vector3(2 * w, 0, 0)
)
tran = sim.add_flux(fcen, df, nfreq, tran_fr)

# for normal run, load negated fields to subtract incident from refl. fields
sim.load_minus_flux_data(refl, straight_refl_data)

pt = mp.Vector3(wvg_xcen, 0.5 * sy - dpml - 0.5)

sim.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, pt, 1e-3))

bend_refl_flux = mp.get_fluxes(refl)
bend_tran_flux = mp.get_fluxes(tran)

flux_freqs = mp.get_flux_freqs(refl)

wl = []
Rs = []
Ts = []
for i in range(nfreq):
    wl = np.append(wl, 1 / flux_freqs[i])
    Rs = np.append(Rs, -bend_refl_flux[i] / straight_tran_flux[i])
    Ts = np.append(Ts, bend_tran_flux[i] / straight_tran_flux[i])

if mp.am_master():
    plt.figure()
    plt.plot(wl, Rs, "bo-", label="reflectance")
    plt.plot(wl, Ts, "ro-", label="transmittance")
    plt.plot(wl, 1 - Rs - Ts, "go-", label="loss")
    plt.axis([5.0, 10.0, 0, 1])
    plt.xlabel("wavelength (μm)")
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
import meep as mp
import numpy as np
from typing import List

# set data for wavelengt and n k materials
wavelengths = dataNK["Wavelength (nm)"]
wavelengthsUm = [i / 1000 for i in wavelengths]
auN = dataNK["Au_n"]
auK = dataNK["Au_k"]
spiroN = dataNK["Spiro_n"]
spiroK = dataNK["Spiro_k"]
perovN = dataNK["Perov_n"]
perovK = dataNK["Perov_k"]
tio2N = dataNK["TiO2_n"]
tio2K = dataNK["TiO2_k"]

# Define the dimensions of the simulation
# 1um = 1000 nm
auHeight = 0.1  # 0.1 um or 100 nm
spiroHeight = 0.2  # 0.2 um or 200 nm
perovHeight = 0.37  # 0.37 um or 370 nm
tio2Height = 0.07  # 0.07 um or 70 nm
x_length = 1  # 1 um or 1000 nm
y_length = 1  # 1 um or 1000 nm

nmtoum = 1000


# Define function to get mediums
def getMediums(wavelengths: List[float], nArray: List[float], kArray: List[float]):
    mediums = []
    for i in range(len(wavelengths)):
        Nindex = complex(nArray[i], kArray[i])
        eps = np.square(Nindex)

        D_conductivity = 2 * np.pi * (1 / (wavelengths[i])) * eps.imag / eps.real
        # print(eps.real, D_conductivity)
        med = mp.Medium(epsilon=eps.real, D_conductivity=D_conductivity)

        mediums.append(med)
        # print(vars(mediums[i]))

    return mediums


# Init list of mediums
auMat = []
spiroMat = []
perovMat = []
tio2Mat = []

# Define materials
auMat = getMediums(wavelengthsUm, auN, auK)
# spiroMat = getMediums(wavelengthsUm, spiroN, spiroK)
# perovMat = getMediums(wavelengthsUm, perovN, perovK)
# tio2Mat = getMediums(wavelengthsUm, tio2N, tio2K)


# Define function to set geometry
def SetGeometry(index):
    geometry = []
    geometry.append(
        mp.Block(mp.Vector3(x_length, y_length, auHeight), material=auMat[index])
    )
    return geometry


geometryAu0 = SetGeometry(0)


# Define the frequency range of the source (in PHz)
fmin = 0.27  # corresponds to 1100nm
fmax = 2.73  # corresponds to 300nm


# Define the spectral profile of the source
def spectral_profile(f):
    if f < fmin or f > fmax:
        return 0
    else:
        # This is a simple uniform spectral profile. You could replace this with a more accurate solar spectral profile.
        return 1


# Define the source
source = mp.Source(
    src=mp.CustomSource(frequency=fmin, width=fmax - fmin, func=spectral_profile),
    component=mp.Ez,  # Assuming Ez polarization, change as needed
    center=mp.Vector3(0, 0, 0),  # Source position, adjust as needed
    size=mp.Vector3(x_length, y_length, 0),  # Source size, adjust as needed
)